In [0]:
%fs mkdirs dbfs:/FileStore/msba_project_data/landing/data_drift

res0: Boolean = true

In [0]:
import pandas as pd
import numpy as np
from sklearn import datasets
from evidently.dashboard import Dashboard  
from evidently.tabs import DataDriftTab
import mlflow
import evidently
import smtplib

In [0]:
%run /Users/abbar005@umn.edu/ml-exp/email-alert-script

In [0]:
cols_of_interest = ["Temperature_F", "Wind_Chill_F", "Humidity_percent", "Pressure_in", "Visibility_mi", "Wind_Speed_mph", "Precipitation_in"]
df = spark.read.table("msba_6331_group1.default.accidents_final").cache()
train, test = df.select(cols_of_interest).randomSplit([0.7, 0.3], 0)
train_s, test_s = train.sample(fraction=0.1, seed=0).toPandas(), test.sample(fraction=0.1, seed=0).toPandas()

In [0]:
outlier_from_train = train.select("Temperature_F", "Wind_Chill_F", "Humidity_percent", "Pressure_in", "Visibility_mi", "Wind_Speed_mph", "Precipitation_in").toPandas()

outlier_df = pd.DataFrame()
num_outliers = int(len(outlier_from_train) * 0.5)  # 50% outliers

for col in outlier_from_train.columns:
    # Calculate mean and standard deviation
    mean = outlier_from_train[col].mean()
    std = outlier_from_train[col].std()

    # Generate normal data
    normal_data = np.random.normal(mean, std, size=len(outlier_from_train) - num_outliers)

    # Generate outlier data
    # Outliers are generated as values far from the mean
    outliers = np.random.normal(mean, std * 3, size=num_outliers)  

    # Combine and shuffle normal data and outliers
    combined_data = np.concatenate([normal_data, outliers])
    np.random.shuffle(combined_data)

    outlier_df[col] = combined_data
test_s = outlier_df

In [0]:
def detect_drift(train_s, test_s):
    # print(train_s.count())
    # print(test_s.count())
    train_pd, test_pd = train_s, test_s
    EXPERIMENT_NAME="/Users/abbar005@umn.edu/data-drift/reports_data_drift"
    
    mlflow.set_experiment(EXPERIMENT_NAME)
    with mlflow.start_run():
        drift_dashboard = Dashboard(tabs=[DataDriftTab])

        drift_dashboard.calculate(train_pd, test_pd)
        drift_dashboard.save(EXPERIMENT_NAME+"/input_data_drift.html")
        drift_dashboard.save("/Users/abbar005@umn.edu/data-drift"+"/input_data_drift.html")
        drift_dashboard._save_to_json(EXPERIMENT_NAME+"/input_data_drift.json")
        mlflow.log_artifacts(EXPERIMENT_NAME)

        json_file = pd.read_json(EXPERIMENT_NAME+"/input_data_drift.json")
        msg = json_file.widgets.values[0]['title']
        send_email_redmail(path=EXPERIMENT_NAME+"/input_data_drift.html", subject=msg)

In [0]:
detect_drift(train_s, test_s)

Temperature_F       3490
Wind_Chill_F        3490
Humidity_percent    3490
Pressure_in         3490
Visibility_mi       3490
Wind_Speed_mph      3490
Precipitation_in    3490
dtype: int64
Temperature_F       35350
Wind_Chill_F        35350
Humidity_percent    35350
Pressure_in         35350
Visibility_mi       35350
Wind_Speed_mph      35350
Precipitation_in    35350
dtype: int64


Uploading artifacts:   0%|          | 0/2 [00:00<?, ?it/s]

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/redmail/email/sender.py:164: FutureWarning:

Argument user_name was renamed as username. Please use username instead.



(552, b"5.3.4 Your message exceeded Google's message size limits. Please visit\n5.3.4  https://support.google.com/mail/?p=MaxSizeError to view our size\n5.3.4 guidelines. pr1-20020a05620a86c100b0077d7cfde46csm5843097qkn.106 - gsmtp", 'abbar005@umn.edu')
sending email without attachments
